In [10]:
# hyperparameter tuning

# Import necessary libraries
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os
import dagshub

In [11]:
mlflow.set_tracking_uri('https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow')
dagshub.init(repo_owner='kunalkaushik1982', repo_name='womens-ecommerce-clothing-reviews', mlflow=True)

Initialized MLflow to track repo "kunalkaushik1982/womens-ecommerce-clothing-reviews"

Repository kunalkaushik1982/womens-ecommerce-clothing-reviews initialized!

In [12]:
reviews=pd.read_csv(r'C:\Users\work\Documents\womens-ecommerce-clothing-reviews\Womens Clothing E-Commerce Reviews.csv')
reviews.drop(['Unnamed: 0', 'Clothing ID','Age','Title','Recommended IND','Positive Feedback Count','Division Name','Department Name','Class Name'], axis=1, inplace=True)
reviews=reviews.dropna(how='any',axis=0)
reviews.head()

,Review Text,Rating
0,Absolutely wonderful - silky and sexy and comf...,4
1,Love this dress! it's sooo pretty. i happene...,5
2,I had such high hopes for this dress and reall...,3
3,"I love, love, love this jumpsuit. it's fun, fl...",5
4,This shirt is very flattering to all due to th...,5


In [13]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\work\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\work\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['Review Text'] = df['Review Text'].apply(lower_case)
        df['Review Text'] = df['Review Text'].apply(remove_stop_words)
        df['Review Text'] = df['Review Text'].apply(removing_numbers)
        df['Review Text'] = df['Review Text'].apply(removing_punctuations)
        df['Review Text'] = df['Review Text'].apply(removing_urls)
        df['Review Text'] = df['Review Text'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [15]:
reviews = normalize_text(reviews)
reviews.head()

,Review Text,Rating
0,absolutely wonderful silky sexy comfortable,4
1,love dress sooo pretty happened find store i m...,5
2,high hope dress really wanted work me initiall...,3
3,love love love jumpsuit fun flirty fabulous ev...,5
4,shirt flattering due adjustable front tie perf...,5


In [16]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(reviews['Review Text'])
y = reviews['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Set the experiment name
mlflow.set_experiment("LoR Hyperparameter Tuning")

<Experiment: artifact_location='mlflow-artifacts:/66dcd983b2fa44a1beaf80e224dd298b', creation_time=1735391499128, experiment_id='3', last_update_time=1735391499128, lifecycle_stage='active', name='LoR Hyperparameter Tuning', tags={}>

In [18]:
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

In [20]:
# Start the parent run for hyperparameter tuning
with mlflow.start_run():

    # Perform grid search
    grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Log each parameter combination as a child run
    for params, mean_score, std_score in zip(grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score'], grid_search.cv_results_['std_test_score']):
        with mlflow.start_run(run_name=f"LR with params: {params}", nested=True):
            model = LogisticRegression(**params)
            model.fit(X_train, y_train)
            
            # Model evaluation
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred,average='weighted')
            recall = recall_score(y_test, y_pred,average='weighted')
            f1 = f1_score(y_test, y_pred,average='weighted')
            
            # Log parameters and metrics
            mlflow.log_params(params)
            mlflow.log_metric("mean_cv_score", mean_score)
            mlflow.log_metric("std_cv_score", std_score)
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)
            
            
            # Print the results for verification
            print(f"Mean CV Score: {mean_score}, Std CV Score: {std_score}")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F1 Score: {f1}")

    # Log the best run details in the parent run
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    mlflow.log_params(best_params)
    mlflow.log_metric("best_f1_score", best_score)
    
    print(f"Best Params: {best_params}")
    print(f"Best F1 Score: {best_score}")

    # Save and log the notebook                
    notebook_path = "exp4_lor_bow_hp.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)

    # Log model
    mlflow.sklearn.log_model(grid_search.best_estimator_, "model")

c:\Users\work\Documents\womens-ecommerce-clothing-reviews\myenv\Lib\site-packages\sklearn\model_selection\_search.py:1107: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  warnings.warn(


Mean CV Score: nan, Std CV Score: nan
Accuracy: 0.6290571870170015
Precision: 0.5653015425860526
Recall: 0.6290571870170015
F1 Score: 0.572825839054155
🏃 View run LR with params: {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'} at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/3/runs/3669c5d3b8584971b732d1185ede0975
🧪 View experiment at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/3
Mean CV Score: nan, Std CV Score: nan
Accuracy: 0.6423051446235372
Precision: 0.5904172312906731
Recall: 0.6423051446235372
F1 Score: 0.6017750908070417
🏃 View run LR with params: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'} at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/3/runs/d855ca37e85a4eeaa33dee89ea847862
🧪 View experiment at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/3
Mean CV Score: nan, Std CV Score: n

2024/12/28 18:54:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run orderly-panda-115 at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/3/runs/427ebec23d9346ddac2a01e8b1b1999a
🧪 View experiment at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/3
